# Please run this .ipynb in Google Colab with an associated Google Drive account. You may want to consult the following two links to get more information about how to get FREE access to these resources.

https://www.google.com/drive/

https://colab.research.google.com/

## (i) Here we install the onnxruntime library

In [ ]:
!pip install onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# (ii) Here we import all necessary libraries

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
import onnxruntime as rt
import numpy as np
import matplotlib.pyplot as plt
import os 
import skimage as ski
import skimage.feature
import matplotlib.pyplot as plt
import nibabel                      as nib
import nibabel.freesurfer.mghformat as mgh


from scipy.ndimage import zoom

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## (iii) Please put all your files at a desired location. Ideally, under "/content/drive/MyDrive/" folder. Just upload all your files to the root folder of Google Drive should work.

## (iv) Preprocessing the data. Convert the mgz format into npy. 

In [ ]:

brains=[]
path="" # path to the folder (brain.mgz)
files=os.listdir(path)
for file in files:
  mgh_file = mgh.load(path+'/'+file)# for each mgz file
  new_array = zoom(mgh_file.get_data(), (0.5, 0.5, 0.5))
  brains.append(new_array)
brains=np.asarray(brains)
# rescale brain matrices from 128*128*128 to 82*86*100
X=brains
coord=[]
x_range=0
y_range=0
z_range=0
for i in range(X.shape[0]):
  buf=X[i, :, :, :]
  buf=buf.reshape((128,128,128))
  xmin=0
  xmax=0
  ymin=0
  ymax=0
  zmin=0
  zmax=0
  for xm in range(0, 128):
    if np.sum(buf[xm, :, :])>50:
      xmin=xm
      break
  for xm in range(0, 128):
    if np.sum(buf[127-xm, :, :])>50:
      xmax=127-xm
      break
  for ym in range(0,128):
    if np.sum(buf[:, ym, :])>50:
      ymin=ym
      break
  for ym in range(0, 128):
    if np.sum(buf[:, 127-ym, :])>50:
      ymax=127-ym
      break
  for zm in range(0, 128):
    if np.sum(buf[:, :, zm])>50:
      zmin=zm
      break
  for zm in range(0, 128):
    if np.sum(buf[:, :, 127-zm])>50:
      zmax=127-zm
      break
  td=[]
  td.append(abs(xmax+xmin)/2)
  td.append(abs(ymax+ymin)/2)
  td.append(abs(zmax+zmin)/2)
  coord.append(td)

x_range=82
y_range=86
z_range=100
date1=[]
data_new=[]
label=[]
id_list=[]
for i in range(X.shape[0]):
  buf=X[i, :, :, :]
  co=coord[i]
  buf=buf.reshape((128,128,128))
  data_new.append(buf[int(int(co[0])-(x_range/2)):int(int(co[0])+(x_range/2)), int(int(co[1])-(y_range/2)):int(int(co[1])+(y_range/2)), int(int(co[2])-(z_range/2)):int(int(co[2])+(z_range/2))])
np.save("brains.npy", data_new) # save the processed data



## (v) Load all necessary files and use our models to make prediction

In [ ]:
providers = ['CPUExecutionProvider']
output_path = "/content/drive/MyDrive/3dcnn.onnx" #Pls change the path and load our onnx model
output_names = ['dense_1']
m = rt.InferenceSession(output_path, providers=providers)
x = np.load("brains.npy")# This is the path for the input data. 
onnx_pred = m.run(output_names, {"input": x})
onnex_pred=onnx_pred[0] + 22 # This is predicted BA. There is a bias of 22 to train the model. Now we add it back.
print(onnx_pred[0])